In [1]:
api_key = "YOUR_DEXTOOLS_API_KEY"
from dextools_python import DextoolsAPIV2
dextools = DextoolsAPIV2(api_key, plan="trial")
import time
import pandas as pd


**SET UP COIN KEYS AND VALUES**

In [2]:
def create_dict_from_file(file_path):
    coin_dict = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
        for i in range(0, len(lines), 2):
            key = lines[i].strip()
            if i + 1 < len(lines):
                value = lines[i + 1].strip()
                coin_dict[key] = value
    
    return coin_dict

# Example usage
file_path = 'coins.txt'
coin_dict = create_dict_from_file(file_path)
print(coin_dict)

{'BIOS': '0x73Cb479f2ccf77BAd90BcDA91e3987358437240A'}


**CALL API FOR PRICE, 1h, 24h, 6h**

In [3]:
def get_token_prices(coin_dict):
    price_dict = {}
    for token_name, pair_address in coin_dict.items():
        price = dextools.get_token_price("base", pair_address)
        if price is not None:
            price_dict[token_name] = price # Add the price dict to the token name key and override the value
        time.sleep(1.5)  # Sleep for 1 second to avoid rate limiting

    # Prepare data for DataFrame
    data = []
    for token_name, price_data in price_dict.items():
        if 'data' in price_data:
            price = price_data['data']['price']
            price1h = price_data['data']['price1h']
            price6h = price_data['data']['price6h']
            price24h = price_data['data']['price24h']
            data.append([token_name, price, price1h, price6h, price24h])
        else:
            data.append([token_name, 'Error', 'Error', 'Error', 'Error'])

    # Create DataFrame
    df = pd.DataFrame(data, columns=['Token', 'Price', 'Price24h', 'Price1h', 'Price6h'])

    # Display the DataFrame
    print(df)
    return df
#dicts= get_token_prices(coin_dict)
#print(dicts)

**SELL TARGETS**

In [4]:
def create_sell_targets_dict(file_path):
    sell_targets = {}
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file if line.strip()]
        for i in range(0, len(lines), 4):
            token_name = lines[i]
            token_address = lines[i + 1]
            price_target = float(lines[i + 2])
            amount_to_sell = float(lines[i + 3])
            sell_targets[token_name] = {
                'address': token_address,
                'price_target': price_target,
                'amount_to_sell': amount_to_sell
            }
    return sell_targets

# Example usage
file_path = 'sell_targets.txt'
sell_targets = create_sell_targets_dict(file_path)
print(sell_targets)

{'BIOS': {'address': '0x73Cb479f2ccf77BAd90BcDA91e3987358437240A', 'price_target': 0.06, 'amount_to_sell': 0.5}, 'DEGENC': {'address': '0x99298c6be0e8ec9e56b7a2be5850abe1fc109d94', 'price_target': 0.013, 'amount_to_sell': 1.0}, 'WHIM': {'address': '0x39a1cce09d7354ac2db86c6b02924360a10e4793', 'price_target': 0.02, 'amount_to_sell': 0.5}, 'WHIMI': {'address': '0x39a1cce09d7354ac2db86c6b02924360a10e4793', 'price_target': 0.04, 'amount_to_sell': 1.0}, 'VIRTUAL': {'address': '0x0b3e328455c4059EEb9e3f84b5543F74E24e7E1b', 'price_target': 6.0, 'amount_to_sell': 1.0}, 'ARBUS': {'address': '0xBDC27118Ca76B375C6887b0ff068aFb03DfC21A0', 'price_target': 0.035, 'amount_to_sell': 1.0}, 'NOTHING': {'address': '0x410DbC28C81F26c9d026d2D91f27930Eef4d98b9', 'price_target': 0.00225, 'amount_to_sell': 1.0}, 'NANCY': {'address': '0x0b1D66d2B66Eeca25Bc489062fDc362e3c214F81', 'price_target': 0.006, 'amount_to_sell': 1.0}, 'Capminal BUY if 1h close candle above this lvl': {'address': '0xbfa733702305280F066D47

## Get Send price to telegram in a table

In [5]:
# Function to get token prices and send them to Telegram
def get_token_prices_and_send(coin_dict, bot_token, chat_id):
    price_dict = {}
    for token_name, pair_address in coin_dict.items():
        price_data = dextools.get_token_price("base", pair_address)
        if price_data is not None:
            price_dict[token_name] = price_data  # Store price data
        else:
            print(f"Failed to fetch price data for {token_name}")
        time.sleep(1.5)  # Sleep to avoid rate limiting

    # Prepare data for DataFrame
    data = []
    for token_name, price_data in price_dict.items():
        if 'data' in price_data:
            price = price_data['data'].get('price', 'Error') # Default to 'Error' if not found
            price1h = price_data['data'].get('price1h', 'N/A')  # Default to 'N/A'
            price6h = price_data['data'].get('price6h', 'N/A')  # Default to 'N/A'
            price24h = price_data['data'].get('price24h', 'N/A') # Default to 'N/A'
            data.append([token_name, price, price1h, price6h, price24h])
        else:
            data.append([token_name, 'Error', 'N/A', 'N/A', 'N/A']) # Ensure 'N/A' for missing time-based prices

    # Create DataFrame
    df = pd.DataFrame(data, columns=['Token', 'Price', 'Price1h', 'Price6h', 'Price24h'])

    # Convert DataFrame to Markdown-style table
    # Ensure all price fields are treated as strings for consistent formatting
    df_string = (
        f"{'Token':<10} {'Price':<8} {'Price1h':<8} {'Price6h':<8} {'Price24h':<8}\n"
        + "\\n".join(
            f"{str(row['Token']):<10} {str(row['Price']):<8} {str(row['Price1h']):<8} {str(row['Price6h']):<8} {str(row['Price24h']):<8}"
            for _, row in df.iterrows()
        )
    )

    # Send the table
    message = f"```\\n{df_string}\\n```"
    success = send_telegram_notification(bot_token, chat_id, message)
    if success:
        print("DataFrame sent to Telegram successfully.")
    else:
        print("Failed to send DataFrame to Telegram.")


**BUY SIGNAL, ENTRIES**

In [6]:
import time
import pandas as pd

# Function to create entry targets dictionary
def create_entry_targets_dict(file_path):
    entry_targets = {}
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file if line.strip()]
        for i in range(0, len(lines), 3):
            token_name = lines[i]
            token_address = lines[i + 1]
            price_target = float(lines[i + 2])
            entry_targets[token_name] = {
                'address': token_address,
                'price_target': price_target
            }
    return entry_targets

# Load entry targets
# file_path = 'entry.txt'
# entry_targets = create_entry_targets_dict(file_path)

# # Get current prices
# price_dict = {}
# for token_name, info in entry_targets.items():
#     pair_address = info['address']
#     price_data = dextools.get_token_price("base", pair_address)
#     if price_data is not None:
#         price_dict[token_name] = price_data  # Store the price data
#     time.sleep(1.7)  # Sleep to avoid rate limiting

# # Prepare data for DataFrame
# data = []
# for token_name, info in entry_targets.items():
#     if token_name in price_dict and 'data' in price_dict[token_name]:
#         current_price = price_dict[token_name]['data']['price']
#         target_price = info['price_target']
#         # Calculate percentage decrease needed
#         if current_price > 0:
#             percent_decrease = ((current_price - target_price) / current_price) * 100
#         else:
#             percent_decrease = 'N/A'
#         buy_now = int(current_price <= target_price)
#         data.append([
#             token_name,
#             current_price,
#             target_price,
#             f"{percent_decrease:.2f}%",
#             buy_now
#         ])
#     else:
#         data.append([token_name, 'Error', info['price_target'], 'N/A', 'N/A'])

# # Create DataFrame
# df = pd.DataFrame(data, columns=[
#     'Token',
#     'Current Price',
#     'Target Price',
#     '% Decrease Needed',
#     'Buy Now'
# ])

# # Display the DataFrame
# print(df)

**TELEGRAM BOT FUNCTIONS**

In [7]:
import time
import pandas as pd
import requests  # Add this import
# Function to send Telegram notifications
def send_telegram_notification(bot_token, chat_id, message):
    url = f'https://api.telegram.org/bot{bot_token}/sendMessage'
    data = {
        'chat_id': chat_id,
        'text': message,
        'parse_mode': 'Markdown'  # Enables Markdown formatting in messages
    }
    response = requests.post(url, data=data)
    return response.status_code == 200

# Telegram credentials
bot_token = '7609328133:AAGFBpNjTnFsXa3CeGqBJ8zxhBpbJTA_1lo' 
#chat_id = '-1001911294807' #group chat cryptoboys
chat_id = '5641871015'  # private chat with your bot

In [8]:
send_telegram_notification(bot_token, chat_id, 'Hello from Python!')

True

In [9]:
# Function to create sell_targets dictionary
def create_sell_targets_dict(file_path):
    sell_targets = {}
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file if line.strip()]
        for i in range(0, len(lines), 4):
            token_name = lines[i]
            token_address = lines[i + 1]
            price_target = float(lines[i + 2])
            amount_to_sell = float(lines[i + 3])
            sell_targets[token_name] = {
                'address': token_address,
                'price_target': price_target,
                'amount_to_sell': amount_to_sell
            }
    return sell_targets

# Load sell targets
file_path = 'sell_targets.txt'

In [10]:
# Function to get current prices for tokens
def get_current_prices(sell_targets, dextools):
    price_dict = {}
    for token_name, info in sell_targets.items():
        pair_address = info['address']
        price_data = dextools.get_token_price("base", pair_address)
        if price_data is not None:
            price_dict[token_name] = price_data  # Store the price data
        time.sleep(1.5)  # Sleep to avoid rate limiting
    return price_dict

In [11]:

# Function to process tokens and check for sell signals
def process_sell_signals(sell_targets, price_dict, bot_token, chat_id):
    data = []
    for token_name, info in sell_targets.items():
        if token_name in price_dict and 'data' in price_dict[token_name]:
            current_price = price_dict[token_name]['data']['price']
            target_price = info['price_target']
            if current_price > 0:
                percent_increase = ((target_price - current_price) / current_price) * 100
            else:
                percent_increase = 'N/A'
            sell_now = int(current_price >= target_price)
            amount_to_sell_percent = info['amount_to_sell'] * 100

            data.append([
                token_name,
                current_price,
                target_price,
                f"{percent_increase:.2f}%",
                f"{amount_to_sell_percent:.2f}%",
                sell_now
            ])

            if sell_now == 1:
                message = (
                    f"🚀 *Price Target Hit for {token_name}!*\n"
                    f"Current Price: *${current_price:.6f}*\n"
                    f"Target Price: *${target_price:.6f}*\n"
                    f"Amount to Sell: *{amount_to_sell_percent:.2f}%*"
                )
                success = send_telegram_notification(bot_token, chat_id, message)
                if success:
                    print(f"Notification sent for {token_name}.")
                else:
                    print(f"Failed to send notification for {token_name}.")
        else:
            data.append([token_name, 'Error', info['price_target'], 'N/A', 'N/A', 'N/A'])
    return pd.DataFrame(data, columns=[
        'Token',
        'Current Price',
        'Target Price',
        '% Increase Needed',
        'Amount to Sell (%)',
        'Sell Now'
    ])

**TELEGRAM BOT ENTIRE LOOP NEW**

In [12]:
import time


# Main loop to monitor buy signals
def monitor_sell_signals(file_path, dextools, bot_token, chat_id, hours):
    sell_targets = create_sell_targets_dict(file_path)
    start_time = time.time()
    interval = 1 * 60  # 30 minutes in seconds
    duration = hours * 3600  # Total duration in seconds

    while time.time() - start_time < duration:
        print(f"Checking for sell signals at {time.ctime()}")
        price_dict = get_current_prices(sell_targets, dextools)
        df = process_sell_signals(sell_targets, price_dict, bot_token, chat_id)
        print(df)
        print("Waiting for the next check...")
        time.sleep(interval)

**TELEGRAM BUY SIGNAL BOT FUNCTIONS**

In [13]:
# Function to create entry_targets dictionary
def create_entry_targets_dict(file_path):
    entry_targets = {}
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file if line.strip()]
        for i in range(0, len(lines), 3):
            token_name = lines[i]
            token_address = lines[i + 1]
            price_target = float(lines[i + 2])
            entry_targets[token_name] = {
                'address': token_address,
                'price_target': price_target
            }
    return entry_targets

In [14]:
def get_current_prices_entry(targets, dextools):
    price_dict = {}
    for token_name, info in targets.items():
        pair_address = info['address']
        price_data = dextools.get_token_price("base", pair_address)
        if price_data is not None:
            price_dict[token_name] = price_data  # Store the price data
        else:
            print(f"Failed to fetch price data for {token_name}")
        time.sleep(1.7)  # Sleep to avoid rate limiting
    return price_dict

In [15]:
# Function to process buy signals
def process_buy_signals(entry_targets, price_dict, bot_token, chat_id):
    data = []
    for token_name, info in entry_targets.items():
        if token_name in price_dict and 'data' in price_dict[token_name]:
            current_price = price_dict[token_name]['data']['price']
            target_price = info['price_target']
            if current_price > 0:
                percent_decrease = ((current_price - target_price) / current_price) * 100
            else:
                percent_decrease = 'N/A'
            buy_now = int(current_price <= target_price)
            data.append([
                token_name,
                current_price,
                target_price,
                f"{percent_decrease:.2f}%",
                buy_now
            ])

            if buy_now == 1:
                message = (
                    f"🛒 *Buy Signal for {token_name}!*\n"
                    f"Current Price: *${current_price:.6f}*\n"
                    f"Target Price: *${target_price:.6f}*\n"
                    f"% Decrease Needed: *{percent_decrease:.2f}%*"
                )
                success = send_telegram_notification(bot_token, chat_id, message)
                if success:
                    print(f"Notification sent for {token_name}.")
                else:
                    print(f"Failed to send notification for {token_name}.")
        else:
            data.append([token_name, 'Error', info['price_target'], 'N/A', 'N/A'])
            print(f"No price data for {token_name}")
    return pd.DataFrame(data, columns=[
        'Token',
        'Current Price',
        'Target Price',
        '% Decrease Needed',
        'Buy Now'
    ])


# Run buy only

In [16]:
#Modify the combined below


# Both sell and buy combined

In [17]:

# Combined main loop
def monitor_signals(sell_file_path, entry_file_path, dextools, bot_token, chat_id, hours):
    sell_targets = create_sell_targets_dict(sell_file_path)
    entry_targets = create_entry_targets_dict(entry_file_path)
    start_time = time.time()
    interval = 5 * 60  # 30 minutes in seconds
    duration = hours * 3600  # Total duration in seconds

    while time.time() - start_time < duration:
        # print(f"checking price at {time.ctime()}")
        # get_token_prices_and_send(coin_dict, bot_token, chat_id)

        print(f"Checking sell signals at {time.ctime()}")
        sell_price_dict = get_current_prices(sell_targets, dextools)
        sell_df = process_sell_signals(sell_targets, sell_price_dict, bot_token, chat_id)
        print(sell_df)

        print(f"Checking buy signals at {time.ctime()}")
        time.sleep(3)
        entry_price_dict = get_current_prices(entry_targets, dextools)
        buy_df = process_buy_signals(entry_targets, entry_price_dict, bot_token, chat_id)
        print(buy_df)

        print("Waiting for the next check...")
        time.sleep(interval)


**USE TELEGRAM BOT BOTH BUY AND SELL SIGNALS**

In [ ]:
bot_token = 'YOUR_TELEGRAM_BOT_TOKEN'
#chat_id = 'YOUR_GROUP_CHAT_ID'  # group
chat_id = 'YOUR_CHAT_ID'  # private chat with your bot
sell_file_path = 'sell_targets.txt'
entry_file_path = 'entry.txt'

monitor_signals(sell_file_path, entry_file_path, dextools, bot_token, chat_id, hours=12)

Checking sell signals at Tue Jun 10 21:52:10 2025
                                            Token  Current Price  \
0                                            BIOS       0.024265   
1                                          DEGENC       0.003729   
2                                            WHIM       0.002000   
3                                           WHIMI       0.002000   
4                                         VIRTUAL       2.061329   
5                                           ARBUS       0.011297   
6                                         NOTHING       0.000505   
7                                           NANCY       0.001811   
8  Capminal BUY if 1h close candle above this lvl       0.000900   

   Target Price % Increase Needed Amount to Sell (%)  Sell Now  
0       0.06000           147.27%             50.00%         0  
1       0.01300           248.66%            100.00%         0  
2       0.02000           899.82%             50.00%         0  
3       0